In [4]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, SolverFactory, minimize, Suffix

# Define the input parameters in a dictionary
input_params = {
    'demand': 1000,  # in MW
    'costs': {'Coal': 20, 'Gas': 45, 'Renewable': 0},  # Cost in $/MWh
    'emission_factors': {'Coal': 0.9, 'Gas': 0.4, 'Renewable': 0},  # Tons of CO2/MWh
    'emission_cap': 300,  # Total allowed emissions in Tons of CO2
    'capacities': {'Coal': 600, 'Gas': 500, 'Renewable': 400},  # Max capacity in MW
}

def run_model(input_params):

    # Initialize the model
    model = ConcreteModel()

    # Create variables
    power_plants = input_params['costs'].keys()
    model.generation = Var(power_plants, within=NonNegativeReals)

    # Define the objective function
    def cost_rule(m):
        return sum(m.generation[p] * input_params['costs'][p] for p in power_plants)

    model.cost = Objective(rule=cost_rule, sense=minimize)

    # Demand constraint
    def demand_rule(m):
        return sum(m.generation[p] for p in power_plants) == input_params['demand']

    model.demand_constraint = Constraint(rule=demand_rule)

    # Emission constraint
    def emission_rule(m):
        return sum(m.generation[p] * input_params['emission_factors'][p] for p in power_plants) <= input_params['emission_cap']

    model.emission_constraint = Constraint(rule=emission_rule)

    # Capacity constraints for each power plant
    def capacity_rule(m, p):
        return m.generation[p] <= input_params['capacities'][p]

    model.capacity_constraints = Constraint(power_plants, rule=capacity_rule)

    # Choose a solver and solve
    solver = SolverFactory('cplex')


    # Activate dual information
    model.dual = Suffix(direction=Suffix.IMPORT)

    # Solve the model
    solution = solver.solve(model)

    return model, solution, power_plants

model, solution, power_plants = run_model(input_params)



In [ ]:
df.to_csv()

In [8]:
# Log results and duals to W&B
import wandb

# Initialize a new W&B run
wandb.init(project="energy_optimization", name="generation_mix_eeg_seminar", config = input_params)

# After solving the model, log the objective and variable values
wandb.log({"Total Cost": model.cost()})

# Log the decision variable values and other relevant metrics
for p in power_plants:
    wandb.log({
        f'Generation_{p}': model.generation[p].value,
        f'Cost_{p}': model.generation[p].value * input_params['costs'][p],
        f'Emissions_{p}': model.generation[p].value * input_params['emission_factors'][p]
    })

# Log the dual variables
emission_shadow_price = model.dual[model.emission_constraint]
wandb.log({"Emission Shadow Price": emission_shadow_price})

# Finish the W&B run
wandb.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Cost_Coal,▁
Cost_Gas,▁
Cost_Renewable,▁
Emission Shadow Price,▁
Emissions_Coal,▁
Emissions_Gas,▁
Emissions_Renewable,▁
Generation_Coal,▁
Generation_Gas,▁
Generation_Renewable,▁
Total Cost,▁


In [9]:
sweep_config = {
    'method': 'grid',  # Alternatively, 'random' could be used for a non-exhaustive search
    'metric': {
        'name': 'Total Cost',
        'goal': 'minimize'   
    },
    'parameters': {
        'demand': {
            'values': [300, 500, 1000]  # Testing different levels of emission caps
        }
    }
}

# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="energy_optimization")

def sweep_run():
    with wandb.init() as run:
        # Get the emission cap from the current run's configuration
        input_params['demand'] = run.config.demand
        
        # Define and solve the model as before...


        model, solution, power_plants = run_model(input_params)


        # Log the results
        wandb.log({"Total Cost": model.cost()})
        for p in power_plants:
            wandb.log({
                f'Generation_{p}': model.generation[p].value,
                f'Cost_{p}': model.generation[p].value * input_params['costs'][p],
                f'Emissions_{p}': model.generation[p].value * input_params['emission_factors'][p]
            })

        total_cost = model.cost()

        wandb.log({"Total Cost": total_cost})

        

# Run the sweep
wandb.agent(sweep_id, function=sweep_run, count=3)
        




Create sweep with ID: lhfzgpad
Sweep URL: https://wandb.ai/wattcast/energy_optimization/sweeps/lhfzgpad


wandb: Agent Starting Run: sb1wlcwf with config:
wandb: 	demand: 300
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Cost_Coal,▁
Cost_Gas,▁
Cost_Renewable,▁
Emissions_Coal,▁
Emissions_Gas,▁
Emissions_Renewable,▁
Generation_Coal,▁
Generation_Gas,▁
Generation_Renewable,▁
Total Cost,▁▁
Cost_Coal,6000.0


wandb: Agent Starting Run: dsb66tag with config:
wandb: 	demand: 500
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Cost_Coal,▁
Cost_Gas,▁
Cost_Renewable,▁
Emissions_Coal,▁
Emissions_Gas,▁
Emissions_Renewable,▁
Generation_Coal,▁
Generation_Gas,▁
Generation_Renewable,▁
Total Cost,▁▁
Cost_Coal,4000.0


wandb: Agent Starting Run: ez1dyk0r with config:
wandb: 	demand: 1000
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Cost_Coal,▁
Cost_Gas,▁
Cost_Renewable,▁
Emissions_Coal,▁
Emissions_Gas,▁
Emissions_Renewable,▁
Generation_Coal,▁
Generation_Gas,▁
Generation_Renewable,▁
Total Cost,▁▁
Cost_Coal,2222.22222
